In [2]:
import os
import utils
import numpy as np

In [3]:
hap_true_dir = 'org_data/chr22_true.hap.gz'
legend_true_dir = 'org_data/chr22_true.legend.gz'
region_dir = 'org_data/region_1.legend.gz'

In [4]:
def load_lines(filename, header=False):
    header_line = None
    lines = []
    with utils.reading(filename) as fp:
        line_count = 0
        if header:
            header_line = fp.readline().rstrip()
        for line in fp:
            lines.append(line.rstrip())
    return header_line, np.array(lines)


In [5]:
_, hap_lines = load_lines(hap_true_dir)
legend_header, legend_lines = load_lines(legend_true_dir, True)
region_header, region_lines = load_lines(region_dir, True)

legend_header, region_header = legend_header.split(), region_header.split()

legend_position_col = utils.get_item_col(legend_header, 'position', legend_true_dir)
print(legend_position_col)
region_position_col = utils.get_item_col(region_header, 'position', region_dir)
region_flag_col = utils.get_item_col(region_header, 'array_marker_flag', region_dir)


1


In [6]:
hap_lines_split = np.array([(line.split()) for line in hap_lines])
# hap_lines_split.shape
# len(hap_lines_split)

In [7]:
legend_positions_list = np.array([int(line.split()[legend_position_col]) for line in legend_lines])
legend_positions_list.shape


(1109368,)

In [8]:
def one_hot(allele, a1_freq):
    if allele is None:
        return [1.0 - a1_freq, a1_freq]
    return [1 - allele, allele]

In [10]:
label_fw = []
legend_lines_split = np.array([(line.split()) for line in legend_lines])
for line in region_lines:
    items = line.rstrip().split()
    position = int(items[region_position_col])
    if int(items[region_flag_col]) == 0:
        check_array = legend_positions_list == position
        if np.sum(check_array) == 1:
            list_allel_one_hot = []
            for allele in hap_lines_split[check_array].astype(int)[0]:
                list_allel_one_hot.append(one_hot(allele, None))
            label_fw.append(list_allel_one_hot)
        else:
            ids = legend_lines_split[check_array][:, 0]
            for index, id in enumerate(ids):
                temp_id = id.split(":")
                if temp_id[2] == items[2] and temp_id[3] == items[3]:
                    print(index, id, items[2], items[3], temp_id[2], temp_id[3])
                    list_allel_one_hot = []
                    for allele in hap_lines_split[check_array].astype(int)[index]:
                        list_allel_one_hot.append(one_hot(allele, None))
                    label_fw.append(list_allel_one_hot)

0 22:16051453:A:C A C A C
1 22:16061250:T:C T C T C
1 22:16062095:GT:G GT G GT G
0 22:16123252:T:A T A T A
1 22:16123252:T:G T G T G
1 22:16193637:C:T C T C T
0 22:16202604:G:A G A G A
1 22:16202725:C:T C T C T
1 22:16252594:G:T G T G T
0 22:16287538:G:A G A G A
1 22:16287538:G:C G C G C
0 22:16287739:G:A G A G A
0 22:16404838:GA:GAA GA GAA GA GAA
1 22:16404838:GA:G GA G GA G
0 22:16405003:G:C G C G C
1 22:16405003:G:T G T G T
0 22:16446304:G:A G A G A
1 22:16459639:AATAT:A AATAT A AATAT A
1 22:16577044:TG:T TG T TG T
1 22:16581168:G:T G T G T
1 22:16592392:G:GAT G GAT G GAT
1 22:16615065:G:T G T G T
1 22:16618547:G:T G T G T
1 22:16619758:C:G C G C G
0 22:16658055:T:A T A T A
0 22:16663541:C:G C G C G
1 22:16663541:C:T C T C T
0 22:16663648:G:A G A G A
1 22:16663648:G:C G C G C
0 22:16667843:A:C A C A C
1 22:16667843:A:T A T A T
0 22:16687910:CT:CTT CT CTT CT CTT
1 22:16848573:C:T C T C T
0 22:16851461:C:A C A C A
1 22:16851461:C:T C T C T
0 22:16852708:A:C A C A C
0 22:16854058:C:G C

In [21]:
a = np.array(label_fw).reshape(2,-1,2)
a[0]

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [78]:
check_array = legend_positions_list == 16050115
print(one_hot(hap_lines_split[check_array][0][0].astype(int),None))
print(hap_lines_split[check_array][0])
for allele in hap_lines_split[check_array][0].astype(int):
    allele_tmp = one_hot(allele,None)
    print(allele_tmp)
one_hot(hap_lines_split[check_array][0].astype(int),None)


[1, 0]
['0' '0']
[1, 0]
[1, 0]


[array([1, 1]), array([0, 0])]